## 最終課題

In [10]:
# Cell 1: 必要ライブラリのインポート
import requests
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin, urlparse, urlunparse
import json
import time

In [13]:
# Cell 2: 設定
url = "https://www.musashino-u.ac.jp/"  # 最初のURL（トップページ）
domain = "musashino-u.ac.jp"            # ドメイン（サブドメインも許可）

# 除外する拡張子（最小セット）
EXCLUDED_EXTENSIONS = (
    ".pdf", ".jpg", ".jpeg", ".png", ".gif", ".svg",
    ".mp4", ".zip", ".css", ".js", ".xml"
)

# 結果と訪問管理
result = {}      # {URL: <title>文字列}
to_visit = [url] # これから訪問するページ（キュー）
visited = set()  # 訪問済み

In [14]:
# Cell 3: 最小限のURL正規化
def norm_url(base, href):
    """相対→絶対、#除去、簡易正規化。不正URLはNone（授業範囲の最小対処）"""
    if not href:
        return None
    href = href.strip()
    # ナビゲーションにならないスキームは除外
    if href.startswith(("#", "javascript:", "mailto:", "tel:")):
        return None

    try:
        u = urljoin(base, href)  # 相対→絶対
    except Exception:
        return None
    if not u:
        return None

    # フラグメント除去
    u = u.split("#", 1)[0]

    try:
        p = urlparse(u)
    except Exception:
        return None

    scheme = (p.scheme or "").lower()
    netloc = (p.netloc or "").lower()

    # 末尾スラッシュの簡易統一（ルート以外の末尾/は外す）
    path = p.path or "/"
    if path != "/" and path.endswith("/"):
        path = path[:-1]

    # クエリはそのまま（高度な正規化はしない）
    return urlunparse((scheme, netloc, path, "", p.query, ""))

In [ ]:
# Cell 4: クロール本体（ループ）
while to_visit:
    current = to_visit.pop(0)
    if current in visited:
        continue

    try:
        resp = requests.get(current, timeout=10)
        # 既にサーバがcharsetを返している場合は推定しない
        if not resp.encoding:
            resp.encoding = resp.apparent_encoding
    except Exception:
        # エラー時はスキップ
        continue

    visited.add(current)

    # HTML以外はスキップ
    ctype = (resp.headers.get("Content-Type") or "").lower()
    if "text/html" not in ctype:
        continue

    soup = BeautifulSoup(resp.text, "html.parser")

    # HTMLコメント内のリンクは除外（コメントを取り除く）
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        c.extract()

    # <title> の文字列を取得（なければ "No Title"）
    title_tag = soup.find("title")
    title_text = title_tag.get_text(strip=True) if title_tag else "No Title"
    result[current] = title_text

    # ページ内リンクを収集
    for a in soup.find_all("a", href=True):
        full_link = norm_url(current, a["href"])
        if not full_link:
            continue

        # http/https 以外は除外
        if not (full_link.startswith("http://") or full_link.startswith("https://")):
            continue

        # サブドメインも含めて musashino-u.ac.jp のみ許可
        p = urlparse(full_link)
        if not (p.netloc == domain or p.netloc.endswith("." + domain)):
            continue

        # 不要なファイルは除外
        low = full_link.lower()
        if any(ext in low for ext in EXCLUDED_EXTENSIONS):
            continue

        # 未訪問ならキューに追加
        if full_link not in visited and full_link not in to_visit:
            to_visit.append(full_link)

    # サーバーへの配慮：少し待つ（礼儀）
    time.sleep(0.1)


In [ ]:
# Cell 5: 出力（整形表示 + 辞書そのものの表示）
print(json.dumps(result, ensure_ascii=False, indent=2, sort_keys=True))
print(result)  # 「辞書オブジェクトそのものを print」